In [1]:
import pandas as pd

In [2]:
import regex as re

from itertools import chain

from collections import Counter

In [3]:
def ngramize(x, n=3):
    return [
    ''.join(x[i:i+n] )
    for i in range(
    len(x)-n+1
    )
        ]

In [4]:
def count_char_gram(words_set, n=2):
    result = dict(Counter(list(chain(*[ngramize(word, n) for word in set_words]))).most_common())
    result = pd.DataFrame(result.items()).rename(columns={0: 'N_CHAR', 1: 'FREQ'})
    return result

In [5]:
with open('../DATASETS/Quran_Aya_Per_Line.txt', encoding='utf-8') as f:
    lines = [line.strip() for line in f.readlines()]
    words = [[word.strip() for word in line.split()] for line in lines]
    words = set(chain(*words))

In [6]:
with open('../DATASETS/Quran_Aya_Per_Line.txt', encoding='utf-8') as f:
    ar_normal_char = set(f.read())
    ar_normal_char = ''.join(ar_normal_char).strip()
    ar_normal_char = re.sub('\s', '', ar_normal_char)

In [7]:
ar_normal_char

'كأءهشثزصطعةوايآلفبغتضسحقئنرذمىؤإظدجخ'

# UN CORPUS

In [8]:
un_text = pd.read_pickle(r"C:\Users\gherb\Jupyter_Notebooks\[30-08-2021]_UN_Corpus_Translation\UN_2020_04.pkl")

In [9]:
un_text = un_text.sample(200_000).reset_index(drop=True)

In [10]:
un_text = un_text['ar'.split()].reset_index(drop=True)

In [11]:
un_text['LEN'] = un_text['ar'].apply(lambda x: len(x.split()))

In [18]:
list_un_words = list(chain(*[word.split() for word in list(un_text['ar'])]))

In [19]:
list_un_words = [re.sub(f'[^{ar_normal_char}]+', '', word) for word in list_un_words]

In [21]:
list_un_words = [word for word in list_un_words if len(word)>0]

In [25]:
dict_un_words = dict(Counter(list_un_words).most_common(2*len(words)))

In [28]:
set_un_words = set(i for i in dict_un_words if len(i)>0)

In [30]:
len(set_un_words), len(words)

(29740, 14870)

# END OF UN CORPUS

In [31]:
def get_percentile_ranges(df_, n_pcent=10):
    n_factor = n_pcent / 100
    n_range = int((n_factor)**(-1))
    dict_percentiles_ranges = dict(df_.FREQ.describe(percentiles=[((i)*n_factor) for i in range(n_range)]).filter(regex="%"))
    dict_percentiles_ranges = {k: round(v, 2) for k,v in dict_percentiles_ranges.items()}
    dict_percentiles_ranges = {int(v):k for k,v in dict_percentiles_ranges.items()}
    return dict_percentiles_ranges

In [32]:
def get_bin(freq_value, df_, n_pcent):
    dict_percentiles_ranges = get_percentile_ranges(df_, n_pcent)
    key = min([i for i in dict_percentiles_ranges.keys() if freq_value <= i], default="COMMON")
    if key != "COMMON":
        return dict_percentiles_ranges[key]
    else:
        return "COMMON"

# df_01: QURAN

In [45]:
df_01 = pd.DataFrame(dict(Counter(list(chain(*[ngramize(word) for word in words]))).most_common()).items()).rename(columns={0:'N_CHAR', 1:'FREQ'})

In [46]:
df_01['BIN_10'] = df_01.FREQ.apply(lambda x: get_bin(x, df_01, n_pcent=10))

In [47]:
df_01['BIN_5'] = df_01.FREQ.apply(lambda x: get_bin(x, df_01, n_pcent=5))

In [48]:
df_01['BIN_25'] = df_01.FREQ.apply(lambda x: get_bin(x, df_01, n_pcent=25))

In [49]:
df_01

,N_CHAR,FREQ,BIN_10,BIN_5,BIN_25
0,وال,354,COMMON,COMMON,COMMON
1,الم,335,COMMON,COMMON,COMMON
2,رون,193,COMMON,COMMON,COMMON
3,ناه,178,COMMON,COMMON,COMMON
4,بال,173,COMMON,COMMON,COMMON
...,...,...,...,...,...
7464,لنز,1,20%,25%,25%
7465,لطه,1,20%,25%,25%
7466,لآل,1,20%,25%,25%
7467,رحك,1,20%,25%,25%


# DF_02: UN CORPUS

In [50]:
df_02 = pd.DataFrame(dict(Counter(list(chain(*[ngramize(word) for word in set_un_words]))).most_common()).items()).rename(columns={0:'N_CHAR', 1:'FREQ'})

In [51]:
df_02['BIN_10'] = df_02.FREQ.apply(lambda x: get_bin(x, df_02, n_pcent=10))

In [52]:
df_02['BIN_5'] = df_02.FREQ.apply(lambda x: get_bin(x, df_02, n_pcent=5))

In [53]:
df_02['BIN_25'] = df_02.FREQ.apply(lambda x: get_bin(x, df_02, n_pcent=25))

In [54]:
df_02

,N_CHAR,FREQ,BIN_10,BIN_5,BIN_25
0,الم,2409,COMMON,COMMON,COMMON
1,وال,2005,COMMON,COMMON,COMMON
2,الت,1165,COMMON,COMMON,COMMON
3,بال,990,COMMON,COMMON,COMMON
4,الا,659,COMMON,COMMON,COMMON
...,...,...,...,...,...
7723,صرن,1,10%,15%,0%
7724,طيف,1,10%,15%,0%
7725,بإغ,1,10%,15%,0%
7726,وعس,1,10%,15%,0%


In [55]:
def create_dict_bin_sets(df_, bin_grp='BIN_10'):
    try:
        result = df_.groupby([bin_grp], as_index=False).agg(
    {
    'N_CHAR': lambda x: set(x.tolist())
    }
    )
        result = dict(zip(result[bin_grp], result['N_CHAR']))
        return result
    
    except Exception as e:
        print(e)
        print(f"""
Make sure df_ is a pandas dataframe; it's type is {type(df_)}
Make sure {bin_grp} in {list(df_.columns)}
        """)

In [71]:
def compare_bins_sets(dict_01, dict_02):
    print("""
The result will be (dict_01 - dict_02) for common keys between the two dictionaries.
Assuming v for both dictionaries of the set type.
""".strip())
    
    keys_list = [k for k in dict_01.keys() if k in dict_02.keys()]
    assert(len(keys_list)>0),'There are no intersection between the two dictionaries on the level of the keys!'
    
    result = {k:(dict_01[k] - dict_02[k]) for k in keys_list}
    result = pd.DataFrame(result.items()).rename(columns={0: 'BIN', 1: 'DIFFERENCE'})
    result['LEN'] = result['DIFFERENCE'].apply(lambda x: len(x))
    
    print(f'Nominal Difference is {result.LEN.sum()}')
    
    return result
    

In [72]:
dict_01 = create_dict_bin_sets(df_01)

In [73]:
dict_02 = create_dict_bin_sets(df_02)

In [74]:
dict_01_minus_dict_02 = compare_bins_sets(dict_01, dict_02)

The result will be (dict_01 - dict_02) for common keys between the two dictionaries.
Assuming v for both dictionaries of the set type.
Nominal Difference is 6574


In [75]:
dict_01_minus_dict_02

,BIN,DIFFERENCE,LEN
0,20%,"{عيف, شعب, أآل, قوذ, بنف, زدر, جعي, طرة, قوس, ...",1875
1,40%,"{اول, مفل, وتض, ثيب, رصر, إدر, ستذ, كضو, هيق, ...",1074
2,50%,"{طبا, بنك, نقم, يكش, وخذ, يخس, يعة, حرب, سأت, ...",771
3,60%,"{قمر, حلل, بثت, ثقف, ننب, منة, مرف, لقص, يرب, ...",505
4,70%,"{أهد, إذن, سبن, يوا, ظنك, طمس, فذو, أوي, هاب, ...",700
5,80%,"{امي, أعد, لدن, أحب, عثن, فظو, بقر, ارى, جكم, ...",648
6,90%,"{دبر, ملي, زيد, إيم, حمت, نتب, زوج, يله, تمس, ...",595
7,COMMON,"{لعب, وقو, فان, ذوا, وأخ, زين, تقو, يهد, كتم, ...",406


# COMPARING UN_COPRUS - Quran:

In [76]:
dict_02_minus_dict_01 = compare_bins_sets(dict_02, dict_01)

The result will be (dict_01 - dict_02) for common keys between the two dictionaries.
Assuming v for both dictionaries of the set type.
Nominal Difference is 5003


In [77]:
dict_02_minus_dict_01

,BIN,DIFFERENCE,LEN
0,20%,"{كشر, حلل, يكش, مؤخ, بؤا, رأى, بدى, قطت, فضو, ...",694
1,40%,"{بنك, إذن, ذوا, يهد, ئعة, عور, صغا, وبط, وحف, ...",408
2,50%,"{إيم, ووظ, ستذ, ئفه, يدل, افذ, أود, وأخ, لدن, ...",725
3,60%,"{آسي, لوض, عيف, إخل, لطع, بوق, ذين, فجر, جعي, ...",734
4,70%,"{وتض, طبا, طيط, محم, مؤش, فان, قيع, أعد, لقص, ...",715
5,80%,"{لعب, أهد, زيد, إدر, منة, يعة, مرف, تقو, زوج, ...",632
6,90%,"{وقو, قيد, يوا, روع, بيق, شعب, زين, هاب, هاد, ...",626
7,COMMON,"{اول, ملي, امي, حتي, يله, قار, يون, حتج, تكا, ...",469


In [79]:
dict_01_minus_dict_02['WORDS'] = dict_01_minus_dict_02['DIFFERENCE'].apply(lambda x: list(chain(*[[word for word in words if sub in word] for sub in x])))

In [80]:
dict_02_minus_dict_01['WORDS'] = dict_02_minus_dict_01['DIFFERENCE'].apply(lambda x: list(chain(*[[word for word in set_un_words if sub in word] for sub in x])))

In [85]:
dict_01_minus_dict_02['WORDS'].iloc[7]

['لعباده',
 'ويلعب',
 'ولعب',
 'يلعبون',
 'لعبادته',
 'ويلعبوا',
 'والعبد',
 'العباد',
 'للعبيد',
 'ونلعب',
 'لعبرة',
 'ولعبد',
 'بالعبد',
 'لعبا',
 'لعبادنا',
 'العبد',
 'لعبادي',
 'بالعباد',
 'لعب',
 'ولعبا',
 'للعباد',
 'والموقوذة',
 'يذوقون',
 'وقول',
 'فذوقوا',
 'وذوقوا',
 'فليذوقوه',
 'الوقود',
 'وقومك',
 'ليذوقوا',
 'وقومهم',
 'سيطوقون',
 'وقولوا',
 'يذوقوا',
 'فذوقوه',
 'وقوم',
 'وتذوقوا',
 'وقومه',
 'وقوموا',
 'وقومهما',
 'موقوتا',
 'موقوفون',
 'وقولهم',
 'وقومها',
 'وقودها',
 'وقود',
 'ذوقوا',
 'فانبذ',
 'فانطلقا',
 'فانكحوهن',
 'فانظروا',
 'فانهار',
 'فانكحوا',
 'يخصفان',
 'الطوفان',
 'فانفروا',
 'فانتصر',
 'فانشزوا',
 'فان',
 'فانصرنا',
 'فانتقمنا',
 'فانظر',
 'فانتبذت',
 'فانتهوا',
 'فانفجرت',
 'فانتهى',
 'فانظري',
 'فانسلخ',
 'فانتشروا',
 'فانطلقوا',
 'فانقلبوا',
 'فانصب',
 'وجفان',
 'فانفذوا',
 'فانبجست',
 'فانتظروا',
 'فانفلق',
 'تؤذوا',
 'وليأخذوا',
 'تنفذوا',
 'ذواتي',
 'وأخذوا',
 'ذواتا',
 'واتخذوا',
 'آذوا',
 'وخذوا',
 'خذوا',
 'يتخذوا',
 'ذوا',
 'تتخذوا',
 'وأوذوا'

In [86]:
dict_02_minus_dict_01['WORDS'].iloc[7]

['وستتناول',
 'متداولة',
 'ومحاولة',
 'وتتناول',
 'لجداول',
 'مداولات',
 'ومحاولات',
 'المداولات',
 'ويحاول',
 'تناوله',
 'والتداول',
 'للمداولات',
 'المتداول',
 'مداولاتنا',
 'ومداولات',
 'طاولة',
 'تتناوله',
 'مداولاتها',
 'ستتناولها',
 'بمحاولة',
 'وتناول',
 'المحاولة',
 'حاولوا',
 'متناول',
 'يحاول',
 'الجداول',
 'التداولي',
 'ومزاولة',
 'محاولة',
 'تناولت',
 'بتناول',
 'وتداولها',
 'تناولتها',
 'وسيتناول',
 'التداول',
 'حاول',
 'يزاول',
 'وتناولت',
 'باولو',
 'يتناوله',
 'مناولة',
 'المقاولين',
 'تتناول',
 'تناولها',
 'ويتناول',
 'والجداول',
 'تناول',
 'يتناول',
 'مداولاته',
 'أتناول',
 'حاولت',
 'تداول',
 'محاولات',
 'وجداول',
 'وتناولها',
 'تتناولها',
 'المقاول',
 'يتناولها',
 'المحاولات',
 'مزاولة',
 'تحاول',
 'لتناول',
 'لمداولات',
 'المتناول',
 'وحاولت',
 'المناولة',
 'يحاولون',
 'التداولية',
 'والمداولات',
 'تناولا',
 'وتحاول',
 'والمقاولين',
 'مقاول',
 'وتداول',
 'نتناول',
 'المتداولة',
 'للتداول',
 'لمحاولة',
 'جداول',
 'شاملين',
 'العمليتين',
 'العاملين',
 'عاملي',
 'للمس

In [87]:
dict_01_minus_dict_02

,BIN,DIFFERENCE,LEN,WORDS
0,20%,"{عيف, شعب, أآل, قوذ, بنف, زدر, جعي, طرة, قوس, ...",1875,"[ضعيفا, شعب, أآلهتنا, والموقوذة, بنفسك, تزدري,..."
1,40%,"{اول, مفل, وتض, ثيب, رصر, إدر, ستذ, كضو, هيق, ...",1074,"[فتطاول, نداولها, المفلحين, المفلحون, وتضحكون,..."
2,50%,"{طبا, بنك, نقم, يكش, وخذ, يخس, يعة, حرب, سأت, ...",771,"[رطبا, حطبا, طباقا, لأصلبنكم, ولأصلبنكم, ابنك,..."
3,60%,"{قمر, حلل, بثت, ثقف, ننب, منة, مرف, لقص, يرب, ...",505,"[للقمر, وقمرا, والقمر, القمر, واحلل, يحلل, أحل..."
4,70%,"{أهد, إذن, سبن, يوا, ظنك, طمس, فذو, أوي, هاب, ...",700,"[بأهدى, أهديكم, أهدى, أهدك, أهدكم, وأهديك, إذن..."
5,80%,"{امي, أعد, لدن, أحب, عثن, فظو, بقر, ارى, جكم, ...",648,"[حامية, وبكلامي, لغلامين, إجرامي, عامين, قوامي..."
6,90%,"{دبر, ملي, زيد, إيم, حمت, نتب, زوج, يله, تمس, ...",595,"[الدبر, دبره, يتدبرون, مدبرا, ليدبروا, أدبر, م..."
7,COMMON,"{لعب, وقو, فان, ذوا, وأخ, زين, تقو, يهد, كتم, ...",406,"[لعباده, ويلعب, ولعب, يلعبون, لعبادته, ويلعبوا..."


In [113]:
quran_words_investigate = dict(Counter(list(chain(*dict_01_minus_dict_02['WORDS'].iloc[3:]))).most_common())

In [114]:
uncorpus_words_investigate = dict(Counter(list(chain(*dict_02_minus_dict_01['WORDS'].iloc[3:]))).most_common())

In [115]:
uncorpus_words_investigate

{'الهيدروفلوروكربون': 11,
 'الكلوروفلوروكربون': 10,
 'جمهوريةالبوليفارية': 10,
 'البشريةمتلازمة': 9,
 'للفيروسالإيدز': 9,
 'الأطرافاجتماع': 9,
 'الإسرائيليون': 9,
 'البرامجالمشاريع': 9,
 'الأوقيانوغرافية': 8,
 'للدبلوماسية': 8,
 'تشيكوسلوفاكيا': 8,
 'تسخينفاليأوسيتيا': 8,
 'الهيدروكربونية': 8,
 'الهيدروكلورية': 8,
 'بالفيروسالإيدز': 8,
 'الفيروسالإيدز': 8,
 'الإنمائيصندوق': 8,
 'جمهوريةالإسلامية': 8,
 'التيسيرالتعاون': 8,
 'الإسرائيلية': 8,
 'والإسرائيليين': 8,
 'الإسرائيليين': 8,
 'للاحتياجات': 8,
 'للاستنتاجات': 8,
 'للاجتماعات': 8,
 'الانكليزية': 7,
 'بالانكليزية': 7,
 'الاشتباكات': 7,
 'للجماهيرية': 7,
 'بأفغانستان': 7,
 'وأوقيانوسيا': 7,
 'الدبلوماسيين': 7,
 'الدبلوماسية': 7,
 'والدبلوماسية': 7,
 'الدومينيكية': 7,
 'الاستهلاكية': 7,
 'الإستراتيجية': 7,
 'الكهربائية': 7,
 'البكتريولوجية': 7,
 'البشريةالإيدز': 7,
 'وأفغانستان': 7,
 'لأفغانستان': 7,
 'للاختصاصات': 7,
 'للفرانكوفونية': 7,
 'للاندوسلفان': 7,
 'الكيميائية': 7,
 'والكيميائية': 7,
 'النيوزيلندية': 7,
 'للانبعاثات': 7,
 'ا

In [118]:
len(quran_words_investigate), len(uncorpus_words_investigate)

(13151, 27521)

In [122]:
#dict_01_minus_dict_02.to_pickle('QURAN_MINUS_UNCORPUS_DIFFERENCE.pkl', protocol=4)

In [121]:
#dict_02_minus_dict_01.to_pickle('UNCORPUS_MINUS_QURAN.pkl', protocol=4)

In [123]:
pd.read_pickle('QURAN_MINUS_UNCORPUS_DIFFERENCE.pkl')

,BIN,DIFFERENCE,LEN,WORDS
0,20%,"{عيف, شعب, أآل, قوذ, بنف, زدر, جعي, طرة, قوس, ...",1875,"[ضعيفا, شعب, أآلهتنا, والموقوذة, بنفسك, تزدري,..."
1,40%,"{اول, مفل, وتض, ثيب, رصر, إدر, ستذ, كضو, هيق, ...",1074,"[فتطاول, نداولها, المفلحين, المفلحون, وتضحكون,..."
2,50%,"{طبا, بنك, نقم, يكش, وخذ, يخس, يعة, حرب, سأت, ...",771,"[رطبا, حطبا, طباقا, لأصلبنكم, ولأصلبنكم, ابنك,..."
3,60%,"{قمر, حلل, بثت, ثقف, ننب, منة, مرف, لقص, يرب, ...",505,"[للقمر, وقمرا, والقمر, القمر, واحلل, يحلل, أحل..."
4,70%,"{أهد, إذن, سبن, يوا, ظنك, طمس, فذو, أوي, هاب, ...",700,"[بأهدى, أهديكم, أهدى, أهدك, أهدكم, وأهديك, إذن..."
5,80%,"{امي, أعد, لدن, أحب, فظو, عثن, ارى, بقر, جكم, ...",648,"[حامية, وبكلامي, لغلامين, إجرامي, عامين, قوامي..."
6,90%,"{دبر, ملي, زيد, إيم, حمت, نتب, زوج, يله, تمس, ...",595,"[الدبر, دبره, يتدبرون, مدبرا, ليدبروا, أدبر, م..."
7,COMMON,"{لعب, وقو, فان, ذوا, وأخ, زين, تقو, يهد, كتم, ...",406,"[لعباده, ويلعب, ولعب, يلعبون, لعبادته, ويلعبوا..."
